In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [ ]:
device = 0 if torch.cuda.is_available() else -1

In [ ]:
llama_models = {
    "Meta-Llama 3 70B Instruct": "meta-llama/Meta-Llama-3-70B-Instruct",
    "Meta-Llama 3 8B Instruct": "meta-llama/Meta-Llama-3-8B-Instruct",
    "Llama 3.1 70B Instruct": "meta-llama/Llama-3.1-70B-Instruct",
    "Llama 3.1 8B Instruct": "meta-llama/Llama-3.1-8B-Instruct",
}

In [ ]:
def load_sentiment_model(model_name):
    """Load the specified Llama sentiment model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    sentiment_analyzer = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=device)
    return sentiment_analyzer

Cache models

In [ ]:
model_cache = {}

In [ ]:
def analyze_sentiment(text, model_choice):
    """Perform sentiment analysis using the selected Llama model."""
    if model_choice not in model_cache:
        model_cache[model_choice] = load_sentiment_model(llama_models[model_choice])
    sentiment_analyzer = model_cache[model_choice]
    result = sentiment_analyzer(text)[0]
    return f"Sentiment: {result['label']} (Confidence: {result['score']:.2f})"

Gradio interface

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Sentiment Analysis with Llama Models</center></h1>")
    model_choice = gr.Dropdown(list(llama_models.keys()), label="Select Llama Model")
    input_text = gr.Textbox(label="Enter text for sentiment analysis", lines=4)
    output_text = gr.Textbox(label="Sentiment Result")
    gr.Button("Analyze").click(analyze_sentiment, [input_text, model_choice], output_text)

In [ ]:
demo.launch()